In [3]:
import cdsapi

dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": [
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "2m_dewpoint_temperature",
        "2m_temperature",
        "mean_sea_level_pressure",
        "sea_surface_temperature",
        "surface_pressure",
        "100m_u_component_of_wind",
        "100m_v_component_of_wind",
        "10m_wind_gust_since_previous_post_processing",
        "boundary_layer_height",
        "convective_available_potential_energy",
        "k_index",
        "total_column_water_vapour",
        "skin_temperature",
        "mean_convective_precipitation_rate",
        "mean_large_scale_precipitation_rate",
        "mean_snowfall_rate",
        "mean_surface_downward_long_wave_radiation_flux",
        "mean_surface_downward_short_wave_radiation_flux",
        "mean_surface_latent_heat_flux",
        "mean_surface_net_long_wave_radiation_flux",
        "mean_surface_net_short_wave_radiation_flux",
        "mean_surface_sensible_heat_flux",
        "mean_total_precipitation_rate",
        "high_cloud_cover",
        "low_cloud_cover",
        "total_cloud_cover",
        "total_column_cloud_liquid_water",
        "evaporation",
        "runoff",
        "surface_runoff",
        "snow_depth",
        "soil_temperature_level_1",
        "volumetric_soil_water_layer_1",
        "vertically_integrated_moisture_divergence"
    ],
    "year": ["2024"],
    "month": ["09"],
    "day": ["01"],
    "time": [
        "00:00", "01:00", "02:00",
        "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00",
        "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00",
        "21:00", "22:00", "23:00"
    ],
    "data_format": "grib",
    "download_format": "unarchived",
    "area": [90, -180, -90, 180],
    'grid:': [1.0, 1.0],

}

client = cdsapi.Client()
client.retrieve(dataset, request).download()


2025-05-05 15:45:57,470 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-05 15:45:57,471 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-05 15:45:57,936 INFO Request ID is 85b92aff-2a01-4e8e-9da6-9696861cca18
2025-05-05 15:45:58,044 INFO status has been updated to accepted
2025-05-05 15:46:12,320 INFO status has been updated to running
2025-05-05 15:47:52,897 INFO status has been updated to successful


'934478f978e51d7cfa65e4efb5a34747.grib'

In [5]:

import cfgrib

datasets = cfgrib.open_datasets("934478f978e51d7cfa65e4efb5a34747.grib")
datasets


c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a

[<xarray.Dataset> Size: 13MB
 Dimensions:              (time: 24, latitude: 181, longitude: 360)
 Coordinates:
     number               int64 8B 0
   * time                 (time) datetime64[ns] 192B 2024-09-01 ... 2024-09-01...
     step                 timedelta64[ns] 8B 00:00:00
     depthBelowLandLayer  float64 8B 0.0
   * latitude             (latitude) float64 1kB 90.0 89.0 88.0 ... -89.0 -90.0
   * longitude            (longitude) float64 3kB -180.0 -179.0 ... 178.0 179.0
     valid_time           (time) datetime64[ns] 192B 2024-09-01 ... 2024-09-01...
 Data variables:
     swvl1                (time, latitude, longitude) float32 6MB ...
     stl1                 (time, latitude, longitude) float32 6MB ...
 Attributes:
     GRIB_edition:            1
     GRIB_centre:             ecmf
     GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
     GRIB_subCentre:          0
     Conventions:             CF-1.7
     institution:             European Centre 

In [6]:
import xarray as xr
import pandas as pd
from functools import reduce



dfs = []

for ds in datasets:
    # Try to drop 'step' and other multi-dim coords if needed
    if "step" in ds.dims:
        ds = ds.mean(dim="step")  # Or choose step=0 or another value

    if "valid_time" in ds.coords:
        ds = ds.drop_vars("valid_time")

    if "number" in ds.coords:
        ds = ds.drop_vars("number")

    try:
        df = ds.to_dataframe().reset_index()
        dfs.append(df)
    except Exception as e:
        print(f"Failed converting a dataset: {e}")

# Merge all DataFrames on ['time', 'latitude', 'longitude']
if dfs:
    merged_df = reduce(lambda left, right: pd.merge(
        left, right,
        on=["time", "latitude", "longitude"],
        how="outer",
        suffixes=('', '_dup')  # Prevent merge error on duplicate names
    ), dfs)

    # Drop duplicated columns if created
    merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_dup')]

    print(merged_df.head())
else:
    merged_df = pd.DataFrame()
    print("No dataframes were created from the datasets.")

                 time  latitude  longitude step  depthBelowLandLayer  swvl1  \
0 2024-08-31 18:00:00     -90.0     -180.0  NaT                  NaN    NaN   
1 2024-08-31 18:00:00     -90.0     -179.0  NaT                  NaN    NaN   
2 2024-08-31 18:00:00     -90.0     -178.0  NaT                  NaN    NaN   
3 2024-08-31 18:00:00     -90.0     -177.0  NaT                  NaN    NaN   
4 2024-08-31 18:00:00     -90.0     -176.0  NaT                  NaN    NaN   

   stl1  surface  sro       fg10  ...  u10  v10  t2m  d2m  lcc  hcc  skt  \
0   NaN      0.0  0.0  15.699446  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1   NaN      0.0  0.0  15.699446  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2   NaN      0.0  0.0  15.699446  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3   NaN      0.0  0.0  15.699446  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4   NaN      0.0  0.0  15.699446  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   u100  v100         kx  
0   NaN   NaN -18.899942  
1   NaN   NaN 

In [7]:
missing_data = merged_df.isnull()
for column in missing_data.columns.values.tolist():
    missing_count = missing_data[column].sum()
    print(f"{column} has {missing_count} missing values")

time has 0 missing values
latitude has 0 missing values
longitude has 0 missing values
step has 65160 missing values
depthBelowLandLayer has 65160 missing values
swvl1 has 65160 missing values
stl1 has 65160 missing values
surface has 1433520 missing values
sro has 1433520 missing values
fg10 has 1433520 missing values
e has 1433520 missing values
ro has 1433520 missing values
vimd has 1433520 missing values
avg_lsprate has 1433520 missing values
avg_cpr has 1433520 missing values
avg_tsrwe has 1433520 missing values
avg_ishf has 1433520 missing values
avg_slhtf has 1433520 missing values
avg_sdswrf has 1433520 missing values
avg_sdlwrf has 1433520 missing values
avg_snswrf has 1433520 missing values
avg_snlwrf has 1433520 missing values
avg_tprate has 1433520 missing values
sst has 595824 missing values
cape has 65160 missing values
tclw has 65160 missing values
sp has 65160 missing values
tcwv has 65160 missing values
sd has 65160 missing values
msl has 65160 missing values
blh has 6

In [8]:
dfm = merged_df.copy()
dfm = dfm.drop(columns=["step", "depthBelowLandLayer"])


In [9]:
dfm.dtypes

time           datetime64[ns]
latitude              float64
longitude             float64
swvl1                 float32
stl1                  float32
surface               float64
sro                   float32
fg10                  float32
e                     float32
ro                    float32
vimd                  float32
avg_lsprate           float32
avg_cpr               float32
avg_tsrwe             float32
avg_ishf              float32
avg_slhtf             float32
avg_sdswrf            float32
avg_sdlwrf            float32
avg_snswrf            float32
avg_snlwrf            float32
avg_tprate            float32
sst                   float32
cape                  float32
tclw                  float32
sp                    float32
tcwv                  float32
sd                    float32
msl                   float32
blh                   float32
tcc                   float32
u10                   float32
v10                   float32
t2m                   float32
d2m       

In [10]:
dfm['coordinates'] = dfm['latitude'].astype(str) + ',' + dfm['longitude'].astype(str)
dfm['time'] = pd.to_datetime(dfm['time'], format='%Y-%m-%dT%H:%M:%S.%fZ')
dfm['day'] = dfm['time'].dt.day
dfm['month'] = dfm['time'].dt.month
dfm['year'] = dfm['time'].dt.year
dfm['hour'] = dfm['time'].dt.hour
dfm['day of the year'] = dfm['time'].dt.dayofyear

In [11]:
new_order = ['time', 'day of the year', 'day', 'month', 'year', 'hour', 'latitude', 'longitude', 'coordinates'] + [col for col in dfm.columns if col not in ['time', 'day', 'month', 'year', 'hour', 'latitude', 'longitude', 'coordinates']]
dfm = dfm[new_order]
#sst is being dropped because it;s going to be analized in a different pipeline
dfm = dfm.drop(columns=["latitude", "longitude", 'sst'])

In [12]:
dfm.sort_values(by='time')

,time,day of the year,day,month,year,hour,coordinates,swvl1,stl1,surface,...,v10,t2m,d2m,lcc,hcc,skt,u100,v100,kx,day of the year
0,2024-08-31 18:00:00,244,31,8,2024,18,"-90.0,-180.0",NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-18.899942,244
43431,2024-08-31 18:00:00,244,31,8,2024,18,"30.0,51.0",NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.351733,244
43432,2024-08-31 18:00:00,244,31,8,2024,18,"30.0,52.0",NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.702738,244
43433,2024-08-31 18:00:00,244,31,8,2024,18,"30.0,53.0",NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.852013,244
43434,2024-08-31 18:00:00,244,31,8,2024,18,"30.0,54.0",NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.026399,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585565,2024-09-01 23:00:00,245,1,9,2024,23,"-30.0,-55.0",0.445103,287.543762,NaN,...,0.362823,286.511536,282.926392,0.962494,0.0,286.427368,5.822861,1.030670,NaN,245
1585566,2024-09-01 23:00:00,245,1,9,2024,23,"-30.0,-54.0",0.453831,287.551575,NaN,...,-0.648895,286.495911,282.949829,0.999512,0.0,286.271118,7.141220,-0.686127,NaN,245
1585567,2024-09-01 23:00:00,245,1,9,2024,23,"-30.0,-53.0",0.451572,288.020325,NaN,...,-1.180145,286.718567,283.100220,1.000000,0.0,286.439087,8.306259,-1.563080,NaN,245
1585554,2024-09-01 23:00:00,245,1,9,2024,23,"-30.0,-66.0",0.036564,298.930481,NaN,...,-1.276825,294.191223,277.285767,0.000000,0.0,292.800415,-8.147842,-2.896088,NaN,245


In [13]:
hour_check = dfm['hour'].unique().tolist()
print("Unique hours in the dataset:", hour_check)

Unique hours in the dataset: [18, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23]


In [14]:
if dfm['day'].iloc[0] != dfm['day'].iloc[-1]:
    dfm.dropna(subset=['t2m'], inplace=True)

dfm

,time,day of the year,day,month,year,hour,coordinates,swvl1,stl1,surface,...,v10,t2m,d2m,lcc,hcc,skt,u100,v100,kx,day of the year
65160,2024-09-01 00:00:00,245,1,9,2024,0,"-90.0,-180.0",0.342225,225.058411,NaN,...,8.898422,237.699966,234.264267,1.0,1.0,236.104401,9.426971,13.220215,NaN,245
65161,2024-09-01 00:00:00,245,1,9,2024,0,"-90.0,-179.0",0.342225,225.058411,NaN,...,8.898422,237.699966,234.264267,1.0,1.0,236.104401,9.426971,13.220215,NaN,245
65162,2024-09-01 00:00:00,245,1,9,2024,0,"-90.0,-178.0",0.342225,225.058411,NaN,...,8.898422,237.699966,234.264267,1.0,1.0,236.104401,9.426971,13.220215,NaN,245
65163,2024-09-01 00:00:00,245,1,9,2024,0,"-90.0,-177.0",0.342225,225.058411,NaN,...,8.898422,237.699966,234.264267,1.0,1.0,236.104401,9.426971,13.220215,NaN,245
65164,2024-09-01 00:00:00,245,1,9,2024,0,"-90.0,-176.0",0.342225,225.058411,NaN,...,8.898422,237.699966,234.264267,1.0,1.0,236.104401,9.426971,13.220215,NaN,245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628995,2024-09-01 23:00:00,245,1,9,2024,23,"90.0,175.0",0.000004,271.617981,NaN,...,-6.815887,272.425598,271.475220,1.0,0.0,271.659790,-7.464249,-8.957611,NaN,245
1628996,2024-09-01 23:00:00,245,1,9,2024,23,"90.0,176.0",0.000004,271.617981,NaN,...,-6.815887,272.425598,271.475220,1.0,0.0,271.659790,-7.464249,-8.957611,NaN,245
1628997,2024-09-01 23:00:00,245,1,9,2024,23,"90.0,177.0",0.000004,271.617981,NaN,...,-6.815887,272.425598,271.475220,1.0,0.0,271.659790,-7.464249,-8.957611,NaN,245
1628998,2024-09-01 23:00:00,245,1,9,2024,23,"90.0,178.0",0.000004,271.617981,NaN,...,-6.815887,272.425598,271.475220,1.0,0.0,271.659790,-7.464249,-8.957611,NaN,245


In [15]:
import datetime as dt
import os

# Define log path
log_path = r"C:\Users\dmoli\Documents\Coding\Weathercast_project\date_log.csv"
dfm['time'] = pd.to_datetime(dfm['time'])
dfm = dfm.sort_values(['coordinates', 'time'])

# Start tracking broken columns
broken_columns = []

# Start log list
log_entries = []

# Check missing data
missing_data = dfm.isnull()

for column in dfm.columns:
    missing_count = missing_data[column].sum()
    if isinstance(missing_count, pd.Series):
        missing_count = missing_count.sum()
    missing_count = int(missing_count)

    if missing_count > 0:
        timestamp = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Columns that can tolerate up to 3 full missing days
        relaxed_cols = [
            'swvl1', 'stl1', 'cape', 'tclw', 'sp', 'tcwv', 'sd', 'msl', 'blh', 'tcc',
            'u10', 'v10', 't2m', 'd2m', 'lcc', 'hcc', 'skt', 'u100', 'v100'
        ]

        if column not in relaxed_cols:
            if missing_count > 1433520:
                log_entries.append([timestamp, column, missing_count, 'too many missing values'])
                broken_columns.append(column)
        else:
            ratio = round(missing_count / 65160, 2)
            if ratio > 3:
                log_entries.append([timestamp, column, missing_count, f'missing > 3 days ({ratio})'])
                broken_columns.append(column)

# Save log if needed
if log_entries:
    log_df = pd.DataFrame(log_entries, columns=["timestamp", "variable", "missing_count", "note"])
    if os.path.exists(log_path):
        log_df.to_csv(log_path, mode='a', header=False, index=False)
    else:
        log_df.to_csv(log_path, index=False)

# ------- Now: Handle 2-3 hours missing data for specific columns ------- #

# Columns that need special NaN handling
target_columns = [
    'cape', 'tclw', 'sp', 'tcwv', 'sd', 'msl', 'blh', 'tcc', 
    'u10', 'v10', 't2m', 'd2m', 'lcc', 'hcc', 'skt', 'u100', 'v100'
]


# Group by coordinate
for coord, group in dfm.groupby('coordinates'):
    for col in target_columns:
        # Select time and the target variable
        sub_df = group[['time', col]].set_index('time')

        # Find missing values
        nan_mask = sub_df[col].isna()
        if nan_mask.sum() > 0:
            nan_times = sub_df[nan_mask].index

            for t in nan_times:
                # Define +/- 2 hours window
                prev_2h = t - pd.Timedelta(hours=2)
                next_2h = t + pd.Timedelta(hours=2)

                window = sub_df.loc[prev_2h:next_2h][col]

                # If 5 expected timestamps and max 2 missing, proceed
                if len(window) == 5 and window.isna().sum() <= 2:
                    # Check if missing hours are consecutive
                    missing_consec = window.isna().astype(int).diff().abs().sum() == 2

                    if missing_consec:
                        # Interpolate over window
                        dfm.loc[(dfm['coordinates'] == coord) & (dfm['time'].between(prev_2h, next_2h)), col] = \
                            dfm.loc[(dfm['coordinates'] == coord) & (dfm['time'].between(prev_2h, next_2h)), col].interpolate(method='linear', limit_direction='both')
                    else:
                        # Take average of 2 hours before and 2 hours after
                        try:
                            before = sub_df.loc[[prev_2h, t - pd.Timedelta(hours=1)]][col].dropna()
                            after = sub_df.loc[[t + pd.Timedelta(hours=1), next_2h]][col].dropna()
                            if len(before) == 2 and len(after) == 2:
                                avg_val = pd.concat([before, after]).mean()
                                dfm.loc[(dfm['coordinates'] == coord) & (dfm['time'] == t), col] = avg_val
                        except Exception as e:
                            print(f"Error filling value for {coord} at {t}:", e)
                            continue
low_freq_columns = [
    'swvl1', 'stl1', 'surface', 'sro', 'fg10', 'e', 'ro', 'vimd',
    'avg_lsprate', 'avg_cpr', 'avg_tsrwe', 'avg_ishf', 'avg_slhtf',
    'avg_sdswrf', 'avg_sdlwrf', 'avg_snswrf', 'avg_snlwrf', 
    'avg_tprate', 'kx'
]

# Sort first to ensure time continuity
dfm = dfm.sort_values(['coordinates', 'time'])

# Interpolate low-frequency columns within each coordinate group
for col in low_freq_columns:
    dfm[col] = dfm.groupby('coordinates')[col].transform(
        lambda group: group.interpolate(method='linear', limit_direction='both')
    )

In [18]:
missing_data1 = dfm.isnull()
for column in missing_data1.columns.values.tolist():
    missing_count = missing_data1[column].sum()
    print(f"{column} has {missing_count} missing values")

time has 0 missing values
day of the year has day of the year    0
day of the year    0
dtype: int64 missing values
day has 0 missing values
month has 0 missing values
year has 0 missing values
hour has 0 missing values
coordinates has 0 missing values
swvl1 has 0 missing values
stl1 has 0 missing values
surface has 0 missing values
sro has 0 missing values
fg10 has 0 missing values
e has 0 missing values
ro has 0 missing values
vimd has 0 missing values
avg_lsprate has 0 missing values
avg_cpr has 0 missing values
avg_tsrwe has 0 missing values
avg_ishf has 0 missing values
avg_slhtf has 0 missing values
avg_sdswrf has 0 missing values
avg_sdlwrf has 0 missing values
avg_snswrf has 0 missing values
avg_snlwrf has 0 missing values
avg_tprate has 0 missing values
cape has 0 missing values
tclw has 0 missing values
sp has 0 missing values
tcwv has 0 missing values
sd has 0 missing values
msl has 0 missing values
blh has 0 missing values
tcc has 0 missing values
u10 has 0 missing values
v

In [19]:
dfm.describe()

,time,day of the year,day,month,year,hour,swvl1,stl1,surface,sro,...,v10,t2m,d2m,lcc,hcc,skt,u100,v100,kx,day of the year
count,1563840,1563840.0,1563840.0,1563840.0,1563840.0,1.563840e+06,1.563840e+06,1.563840e+06,1563840.0,1.563840e+06,...,1.563840e+06,1.563840e+06,1.563840e+06,1.563840e+06,1.563840e+06,1.563840e+06,1.563840e+06,1.563840e+06,1.563840e+06,1563840.0
mean,2024-09-01 11:29:59.999996160,245.0,1.0,9.0,2024.0,1.150000e+01,8.913253e-02,2.819480e+02,0.0,3.006138e-06,...,7.711759e-01,2.816779e+02,2.771606e+02,4.403390e-01,3.488820e-01,2.820115e+02,1.375008e-01,8.612286e-01,4.917096e+00,245.0
min,2024-09-01 00:00:00,245.0,1.0,9.0,2024.0,0.000000e+00,-7.955141e-03,2.156756e+02,0.0,0.000000e+00,...,-2.208437e+01,2.090831e+02,2.060271e+02,0.000000e+00,0.000000e+00,2.084575e+02,-3.542459e+01,-2.840306e+01,-7.997363e+01,245.0
25%,2024-09-01 05:45:00,245.0,1.0,9.0,2024.0,5.750000e+00,-3.512949e-06,2.730734e+02,0.0,0.000000e+00,...,-2.263763e+00,2.729680e+02,2.713666e+02,4.384613e-02,0.000000e+00,2.730069e+02,-5.022934e+00,-3.040283e+00,-8.670514e+00,245.0
50%,2024-09-01 11:30:00,245.0,1.0,9.0,2024.0,1.150000e+01,8.754432e-07,2.866329e+02,0.0,0.000000e+00,...,9.072113e-01,2.860558e+02,2.809878e+02,3.101501e-01,3.652954e-02,2.864849e+02,-1.983643e-04,1.129791e+00,8.026588e+00,245.0
75%,2024-09-01 17:15:00,245.0,1.0,9.0,2024.0,1.725000e+01,1.895259e-01,2.986628e+02,0.0,0.000000e+00,...,4.006058e+00,2.976633e+02,2.920377e+02,9.254456e-01,8.704224e-01,2.983401e+02,5.160275e+00,4.930084e+00,2.179828e+01,245.0
max,2024-09-01 23:00:00,245.0,1.0,9.0,2024.0,2.300000e+01,7.647640e-01,3.308865e+02,0.0,6.561677e-03,...,2.272478e+01,3.183840e+02,3.032428e+02,1.000000e+00,1.000000e+00,3.390916e+02,3.285843e+01,3.005258e+01,4.092910e+01,245.0
std,NaN,0.0,0.0,0.0,0.0,6.922189e+00,1.446774e-01,2.144563e+01,0.0,4.905341e-05,...,5.134203e+00,2.002840e+01,1.954133e+01,3.983463e-01,4.217741e-01,2.119258e+01,7.643231e+00,6.688776e+00,2.169387e+01,0.0


In [ ]:
from scipy.stats import skew, kurtosis
import numpy as np

dfm['2tm'] = (dfm['t2m'] - 273.15).round(2)
dfm['d2m'] = (dfm['d2m'] - 273.15).round(2)
dfm['msl'] = (dfm['msl'] / 100).round(2)
dfm['sp'] = (dfm['sp'] / 100).round(2)
dfm['skt'] = (dfm['skt'] - 273.15).round(2)
dfm['stl1'] = (dfm['stl1'] - 273.15).round(2)
dfm['kx'] = (dfm['kx'] - 273.15).round(2)

# Group and compute statistics
grouped = dfm.groupby('coordinates')
summary_df = grouped.agg({'swvl1': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'stl1': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'surface': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'sro': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True)],
                        'e': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'ro': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'vimd': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'avg_lsprate': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'avg_cpr': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'avg_tsrwe': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'avg_ishf': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var], 
                        'avg_slhtf': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'avg_sdswrf': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'avg_sdlwrf': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'avg_snswrf': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'avg_snlwrf': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'avg_tprate': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'cape': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'tclw': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'sp': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'tcwv': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'sd': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'msl': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'blh': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'tcc': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'u10': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'v10': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        't2m': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'd2m': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'lcc': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'hcc': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'skt': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'u100': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'v100': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
                        'kx': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var]
                        }).reset_index()

summary_df.columns = ['_'.join(col).rstrip('_') for col in summary_df.columns.values]
summary_df['Date'] = df['time'].iloc[0]
summary_df['coordinates'] = df['coordinates'].iloc[0]
summary_df['day of the year'] = df['day of the year'].iloc[0]
summary_df.to_csv('era5_summary_by_coordinate.csv', mode='a', header=None,  index=False)


C:\Users\dmoli\AppData\Local\Temp\ipykernel_16808\4065146535.py:14: FutureWarning: The provided callable <function var at 0x0000025D14EA1EE0> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  summary_df = grouped.agg({'swvl1': ['mean', 'min', 'max', 'std', 'median', lambda x: x.mode().iloc[0], lambda x: x.quantile(0.25), lambda x: x.quantile(0.75), lambda x: x.max() - x.min(), skew, lambda x: kurtosis(x, fisher=True), np.var],
c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\groupby\generic.py:324: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  f = lambda x: func(x, *args, **kwargs)
C:\Users\dmoli\AppData\Local\Temp\ipykernel_16808\4065146535.py:15: RuntimeWarning: Precision loss occurred in moment calculat

KeyboardInterrupt: 

In [23]:
print(dfm.columns)


Index(['time', 'day of the year', 'day', 'month', 'year', 'hour',
       'coordinates', 'swvl1', 'stl1', 'surface', 'sro', 'fg10', 'e', 'ro',
       'vimd', 'avg_lsprate', 'avg_cpr', 'avg_tsrwe', 'avg_ishf', 'avg_slhtf',
       'avg_sdswrf', 'avg_sdlwrf', 'avg_snswrf', 'avg_snlwrf', 'avg_tprate',
       'cape', 'tclw', 'sp', 'tcwv', 'sd', 'msl', 'blh', 'tcc', 'u10', 'v10',
       't2m', 'd2m', 'lcc', 'hcc', 'skt', 'u100', 'v100', 'kx',
       'day of the year', '2tm'],
      dtype='object')


In [18]:
import cdsapi

dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": ["2m_temperature"],
    "year": ["1940"],
    "month": ["01"],
    "day": ["02"],
    "time": ["00:00"],
    "data_format": "grib",
    "download_format": "unarchived",
    "area": [-1, -171, -1, -171]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()


2025-05-19 21:14:00,818 WARNING [2025-05-15T00:00:00] Scheduled maintenance of the Data Stores Cloud Infrastructure - 19 May 2025. Please follow status [here](https://status.ecmwf.int/) or in our [forum](https://forum.ecmwf.int/t/scheduled-maintenance-of-the-data-stores-cloud-infrastructure-19-may-2025/13056/1)
2025-05-19 21:14:00,822 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-19 21:14:00,825 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-19 21:14:01,292 INFO Request ID is e9a6443f-6a86-4d22-ac52-667c67ccbda6
2025-05-19 21:14:01,376 INFO status has been updated to accepted
2025-05-19 21:14:34,084 INFO status has been updated to running
2025-05-19 21:15:16,956 INFO st

'f033d9b9c65507d0c373f69f9b10a6c4.grib'

In [19]:

import cfgrib

datasets = cfgrib.open_datasets("f033d9b9c65507d0c373f69f9b10a6c4.grib")
datasets


c:\Users\dmoli\AppData\Local\Programs\Python\Python313\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


[<xarray.Dataset> Size: 60B
 Dimensions:     ()
 Coordinates:
     number      int64 8B 0
     time        datetime64[ns] 8B 1940-01-02
     step        timedelta64[ns] 8B 00:00:00
     surface     float64 8B 0.0
     latitude    float64 8B -1.0
     longitude   float64 8B -171.0
     valid_time  datetime64[ns] 8B ...
 Data variables:
     t2m         float32 4B ...
 Attributes:
     GRIB_edition:            1
     GRIB_centre:             ecmf
     GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
     GRIB_subCentre:          0
     Conventions:             CF-1.7
     institution:             European Centre for Medium-Range Weather Forecasts]

In [20]:
dataset = datasets[0]
print(dataset)

<xarray.Dataset> Size: 60B
Dimensions:     ()
Coordinates:
    number      int64 8B 0
    time        datetime64[ns] 8B 1940-01-02
    step        timedelta64[ns] 8B 00:00:00
    surface     float64 8B 0.0
    latitude    float64 8B -1.0
    longitude   float64 8B -171.0
    valid_time  datetime64[ns] 8B ...
Data variables:
    t2m         float32 4B ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts


In [21]:
import pandas as pd
dataset = datasets[0]

t2m_dataarray = dataset['t2m']

if t2m_dataarray.ndim == 0:
    # It's a scalar value
    t2m_value = t2m_dataarray.item()  # Or just t2m_dataarray.values
    print(f"The value of t2m is: {t2m_value}")
else:
    # It has dimensions, so we can convert to a DataFrame
    t2m_df = t2m_dataarray.to_dataframe()
    print(t2m_df.head())

The value of t2m is: 299.234619140625
